[Текст ссылки](https://)### Выполнил: <font color='red'>Тижин Матвей Романович</font>

### Тема: Web-сервер для обучения и использования ML-моделей

#### Преподаватели: Роман Ищенко (roman.ischenko@gmail.com) и Илья Склонин

**Дедлайн**: 18.01.2026

**Среда выполнения**: Jupyter Notebook (Python 3.9+)

#### Правила:

Результаты выполнения задания:

- архив со скриптами и файлами Dockerfile, который 1-2 команды позволяет развернуть сервер, решающий поставленные в задании задачи
- Jupyter Notebook, где __весь код__ из скриптов дублируется (1 ячейка - 1 скрипт) с комментарием, содержащим информацию о том, из какого файла взят код и что верхнеуровнево этот код делает

__Максимальное число баллов за задание - 35__.

Готовое задание отправляется на почту преподавателя.

Задание выполняется самостоятельно. Если какие-то студенты будут уличены в списывании, все они автоматически получат за эту работу 0 баллов. Если вы нашли в Интернете какой-то специфичный код, который собираетесь заимствовать, обязательно укажите это в задании - наверняка вы не единственный, кто найдёт и использует эту информацию.

Удалять фрагменты формулировок заданий запрещается.

### Постановка задачи:

**Серверная часть (22 балла):**

- __В данной работе нужно написать многозадачный веб-сервер для обучения и инференса ML моделей. На старте сервер получает на вход (через .env) конфиг, в котором должны быть указаны 3 параметра: путь к директории для сохранения моделей внутри контейнера сервера, число ядер, доступных для обучения и максимальное число моделей, которые могут быть одновременно загружены для инференса.__


- Сервер должен реализовывать следующие методы:
    - `fit(X, y, config)` - обучить модель и сохранить на диск по указанным именем
    - `predict(y, config)` - предсказать с помощью обученной и загруженной модели по её имени
    - `load(config)` - загрузить обученную модель по её имени в режим инференса
    - `unload(config)` - выгрузить загруженную модель по её имени
    - `remove(config)` - удалить обученную модель с диска по её имени
    - `remove_all()` - удалить все обученные модели с диска__


- __Содержимое конфигов и форматы данных предлагается продумать и реализовать самостоятельно__
- __Сервер должен иметь счётчик активных процессов. Максимальное число активных процессов соответствует числу ядер, переданному в конфиге при старте сервиса. Каждое обучение модели запускается в отдельном процессе и до своего завершения потребляет этот процесс. Один процесс всегда остаётся для сервера, в нём же загружаются и работают на инференс обученные модели__
- __Сервер должен корректно обрабатывать все граничные случаи (запуск обучения без свободных ядер, запуск инфренса свыше лимита, запросы с несуществующими именами моделей, запросы с дублирующимися именами моделей)__
- __В реализации должны поддерживаться не менее трёх дискриминативных моделей (т.е. принимающих на вход объекты и метки при обучении и предсказывающих метки для новых объектов)__
- __Сервер должен быть реализован на FastAPI__
- Проект разворачивается с помощью выбранной библиотеки управления виртуальными окружениями и технологии контейнеризации Docker

**Клиентская часть (13 баллов):**

- __Клиентская часть должна демонстрировать работу с реализованным сервером с помощью библиотек requests и aiohttp. Она может быть реализована непосредственно в Jupyter Notebook, с описанием ожидаемого действия, или в отдельном(-ых) скрипте(-ах), с дублированием в Jupyter Notebook (тогда работоспособность в ноутбуке не требуется). Далее описываются отдельные функции:
- Код вызова последовательного вызова обучения как минимум двух (N) различных моделей с таким набором данных и параметрами, чтобы обучение одной модели длилось не менее 60 секунд.
- Код вызова асинхронного вызова обучения как минимум двух различных моделей с демонстрацией, что работа выполняется в два (в N) раза быстрее
- Асинхронный вызов нескольких предсказаний
- Код демонстрации остальных функций сервера (загрузка, выгрузка, удаление)
- Должны обрабатываться ошибки и исключения, возвращаемые сервером


In [1]:
from pydantic import BaseModel, ValidationError

class User(BaseModel):
    id : int
    name : str

User(id=1, name="Ivan")

User(id=1, name='Ivan')

In [6]:
import sys
print(sys.executable)

/home/matvey/task_4/.venv/bin/python3


In [2]:
# server/main.py

import os
import asyncio
from contextlib import asynccontextmanager
from typing import List
from concurrent.futures import ProcessPoolExecutor

import joblib
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from sklearn.linear_model import LogisticRegression

# Импортируем наш объект с настройками из server/config.py
from .config import settings



def _train_model_sync(model_path: str, X: List[List[float]], y: List[int]):
    """
    Синхронная функция, которая выполняет CPU-интенсивную задачу обучения модели.
    Предназначена для запуска в ProcessPoolExecutor.
    """
    try:
        if os.path.exists(model_path):
            return f"Модель по пути '{model_path}' уже существует."

        model = LogisticRegression()    
        model.fit(X, y)
        joblib.dump(model, model_path)
        return None
    except Exception as e:
        return f"Ошибка при обучении модели: {e}"


class FitConfig(BaseModel):
    model_name : str = Field(..., max_length=30, description="Уникальное имя для сохранения модели")

class FitRequest(BaseModel):
    X : List[List[float]]
    y : List[int]
    config: FitConfig



@asynccontextmanager
async def lifespan(app: FastAPI):
    app.state.process_pool = ProcessPoolExecutor(max_workers=settings.TRAINING_CORES)
    print(f"Сервер запущен. Пул процессов на {settings.TRAINING_CORES} воркера(ов) создан.")
    
    yield
    
    app.state.process_pool.shutdown(wait=True)
    print("Пул процессов остановлен.")

app = FastAPI(title="ML Model Server", lifespan=lifespan)


# Эндпоинты API

@app.get('/')
def read_root():
    return {"message": "Добро пожаловать на сервер для ML моделей!"}

@app.post('/fit')
async def fit_model(request: FitRequest):
    
    os.makedirs(settings.MODELS_PATH, exist_ok=True)
    
    model_name = request.config.model_name
    model_path = os.path.join(settings.MODELS_PATH, f'{model_name}.joblib')
    
    loop = asyncio.get_running_loop()
    error_message = await loop.run_in_executor(
        app.state.process_pool,
        _train_model_sync,
        model_path, request.X, request.y
    )

    if error_message:
        raise HTTPException(status_code=400, detail=error_message)

    return {
        "message": "Задача обучения модели запущена в фоновом режиме.",
        "model_name": model_name,
        "path": model_path
    }

ImportError: attempted relative import with no known parent package

In [16]:
model_name = "oanfofn"
model_path = os.path.join(MODELS_DIR, f'{model_name}.joblib')

In [1]:
asyncio.get_running_loop()

NameError: name 'asyncio' is not defined